In [1]:
from umap import UMAP
import h5py
import numpy as np
import torch
import pickle
from IPython import display
from time import time
import pandas as pd
import json
import gc

2022-02-22 20:53:51.245057: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
save_stuff_path = f"/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/SIMULATIONS/UMAP_Regressor/"

# Image directories
GT_file_path_train = f"/home/oiangu/Desktop/Conical_Refraction_Polarimeter/OUTPUT/NOISY/TRAIN/GROUND_TRUTHS.json"
images_dir_path_train =f"/home/oiangu/Desktop/Conical_Refraction_Polarimeter/OUTPUT/NOISY/TRAIN/" 

# Embedder parameters

num_images=2000
umap_dims=100
min_dist=0.1
n_neighbours=700
metric='hamming'
seed=666
decimals=2

embedder_path = save_stuff_path + \
        f'/Embedders/UMAP_EMBEDDER_num_images_{num_images}_umap_dims_{umap_dims}_min_dist_{min_dist}_n_neighbours_{n_neighbours}_metric_{metric}_decimals_{decimals}_seed_{seed}.sav'
#    '/Embedders/SECOND_UMAP_EMBEDDER_num_images_6000_umap_dims_10_min_dist_0.5_n_neighbours_700_metric_hamming_decimals_3_seed_666.sav'


In [3]:
from torch.utils.data import Dataset
from torchvision.io import read_image
from torch.utils.data import DataLoader

class EmbeddingImageDataset(Dataset):
    def __init__(self, GT_file_path, images_dir_path, embedder):
        self.df_GTs = pd.DataFrame.from_dict(json.load(open(GT_file_path)))
        self.images_dir_path = images_dir_path
        self.len_data = len(self.df_GTs)
        self.embedder = embedder
        #self.embdding_dims = embedder.embedding_.shape[-1]

    def __len__(self):
        return self.len_data

    def __getitem__(self, idx):
        img_path = f"{self.images_dir_path}/IM_{self.df_GTs.iloc[idx,0]}_phiCR_{self.df_GTs.iloc[idx,1]}.png"
        image = read_image(img_path) #[1, 2X+1, 2X+1] torch tensor
        label = float(self.df_GTs.iloc[idx, 1])
        return image, label

In [4]:
umap_embedder = UMAP(n_components=umap_dims, min_dist=min_dist, n_neighbors=n_neighbours, metric=metric) #euclidean, canberra, cosine, manhattan, braycurtis, mahalanobis, hamming
umap_embedder = pickle.load((open(embedder_path, 'rb')))

In [5]:
training_data = EmbeddingImageDataset(GT_file_path_train, images_dir_path_train, umap_embedder)

In [6]:
import os
import cv2
image_names = os.listdir(f"{save_stuff_path}/TEST_IMAGES/")
test_h5f = h5py.File(f"{save_stuff_path}/TEST_Embedding_100D_w2000_training.h5", 'a') # append if exists, create if not
for im_name in image_names:
    image = cv2.imread(f"{save_stuff_path}/TEST_IMAGES/{im_name}", cv2.IMREAD_GRAYSCALE)
    test_h5f[im_name] = umap_embedder.transform(np.expand_dims(image.flatten(),0)) #[1, 100]
    test_h5f.flush()

In [6]:
X=302
nims_per_batch=50
number_of_batches=1650*10
np.random.seed(507)
output_info_every=1

beg=time()

# Initialize the hdf5 dataset saver
h5f = h5py.File(f"{save_stuff_path}/Embedding_Datasets/Dataset_Embedding_100D_w2000_training.h5", 'a') # append if exists, create if not
prev_batches = len(h5f)
print(prev_batches)

images = np.zeros((nims_per_batch, (2*X+1)**2), dtype=np.float32)
embedding_and_labels = np.zeros((nims_per_batch, umap_embedder.embedding_.shape[-1]+2), dtype=np.float32)

for batch in range(0,number_of_batches):
    random_indices = np.random.choice(range(len(training_data)), nims_per_batch, replace=False)
    for i, idx in enumerate(random_indices):
        image, label = training_data[idx]
        images[i] = image.flatten()
        embedding_and_labels[i, -2] = label
        embedding_and_labels[i, -1] = idx
    embedding_and_labels[:,:-2] = umap_embedder.transform(images)
    ID=f"{batch+prev_batches}"
    h5f.create_dataset(ID, data=embedding_and_labels, compression="lzf", shuffle=True)
    if batch%output_info_every==0:
        h5f.flush()
        display.clear_output(wait=True)
        elapsed=time()-beg
        print(f"["+'#'*(int(100*batch/number_of_batches))+' '*(100-int(100*batch/number_of_batches))+f"] {100*batch/number_of_batches:3.4}% \n\nSimulated: {batch}/{number_of_batches}\nElapsed time: {elapsed//3600} h {elapsed//60-(elapsed//3600)*60} min {elapsed-(elapsed//60)*60-(elapsed//3600)*60:2.4} s")
        gc.collect()
        
h5f.flush()

[####                                                                                                ] 4.97% 

Simulated: 820/16500
Elapsed time: 20.0 h 30.0 min -1.191e+03 s


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/oiangu/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_124929/2278957344.py", line 24, in <module>
    embedding_and_labels[:,:-2] = umap_embedder.transform(images)
  File "/home/oiangu/anaconda3/lib/python3.8/site-packages/umap/umap_.py", line 2859, in transform
    dmat = pairwise_distances(
  File "/home/oiangu/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/home/oiangu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/pairwise.py", line 1790, in pairwise_distances
    return _parallel_pairwise(X, Y, func, n_jobs, **kwds)
  File "/home/oiangu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/pairwise.py", line 1359, in _parallel_pairwise
    return func(X, Y, **kwds)
  File "/home/oiangu/anaconda3/lib/python3.8/site-packages/s

TypeError: object of type 'NoneType' has no len()